# The type system and multiple dispatch
### Michiel Stock, Bram De Jaegher, Daan Van Hauwermeiren
### 18 December 2019
# Everything has a type

All Julia objects, both those already defined as well as those you might make yourself, have a type. The type system is the secret sauce, allowing Julia to be fast because code can be specialized for a particular combination of types. It is also supremely useful in conjunction with *multiple dispatch*, in which functions work differently depending on which types you feed into them.

# Checking the type

The type of objects can be assessed using the function `typeof`. For collections, `eltype` gives the types of individual elements. Try the following examples. Note that types are always capitalized!

In [1]:
a = 42; s = "mice"; n = 0.9; A = [1 2; 3 4];

In [2]:
typeof(a)

Int64

In [3]:
typeof(s)

String

In [4]:
typeof(n)

Float64

In [5]:
typeof(A)

Array{Int64,2}

These are all *concrete types*. Julia types are part of a hierarchical type system, forming a single, fully connected type graph. The concrete types are the leaves of this tree, whereas the inner nodes are *abstract types*. As hinted by the name, these are abstract and cannot be instantiated. They, however, help with conceptually ordering the type system.

We can find the supertype of a concrete or abstract type using the function `supertype`.

In [6]:
supertype(Int8)

Signed

In [7]:
supertype(Float64)

AbstractFloat

In [8]:
supertype(AbstractFloat)

Real

In [9]:
supertype(Real)

Number

In [10]:
supertype(Number)

Any

In [11]:
supertype(Any)

Any

See how all the numbers are hierarchically represented? Note that any type is always a subtype of `Any`. We can check if an object is (sub)type using the function `isa`.

In [12]:
a isa Int  # Int is concrete type Int64

true

In [13]:
a isa Integer  # Integer is abstract

true

In [14]:
a isa Int8

false

In [15]:
a isa Number


true

Concrete types always have a fixed representation whereas abstract types can be anything. Concrete subtypes of `AbstractFloat` can be 16 bits (`Float16`), 32 bits (`Float32`), 64 bits (`Float64`) or arbitrary large (`BigFloat`).

We can check if one type is a subtype of the second one using the binary operator `<:`.

In [16]:
Float64 <: AbstractFloat

true

In [17]:
Float16 <: AbstractFloat

true

In [18]:
AbstractFloat <: Number

true

In [19]:
Int <: Number

true

In [20]:
Int <: AbstractFloat

false

In [21]:
Integer <: Int

false

# Methods and dispatch

When a function is run for the first time with a particular combination of type inputs, it gets compiled by the LLVM compiler. Such a specific function is referred to as a `method`. Every time a function is run with a new combination of types of arguments, a suitable method is compiled. This is noticeable when measuring the running time. Compare

In [22]:
mynewfun(x) = x^2 + x

A = [1 2; 3 4];

In [23]:
# first run
@time mynewfun(A)

  0.434393 seconds (1.47 M allocations: 70.785 MiB, 5.14% gc time)


2×2 Array{Int64,2}:
  8  12
 18  26

In [24]:
# second run
@time mynewfun(A)

  0.000006 seconds (6 allocations: 384 bytes)


2×2 Array{Int64,2}:
  8  12
 18  26

In [25]:
# now with a float
@time mynewfun(1)

  0.003149 seconds (1.92 k allocations: 107.533 KiB)


2

In [26]:
# new value, same type
@time mynewfun(12)

  0.000002 seconds (4 allocations: 160 bytes)


156

The known methods can be found using the function `methods`. For example, check how many methods there are associated with the humble multiplication operator `*`.

In [27]:
methods(*)

# 339 methods for generic function "*":
[1] *(x::Bool, z::Complex{Bool}) in Base at complex.jl:281
[2] *(x::Bool, y::Bool) in Base at bool.jl:106
[3] *(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:118
[4] *(x::Bool, z::Complex) in Base at complex.jl:288
[5] *(x::Bool, y::AbstractIrrational) in Base at irrationals.jl:135
[6] *(a::Float16, b::Float16) in Base at float.jl:392
[7] *(x::Float32, y::Float32) in Base at float.jl:398
[8] *(x::Float64, y::Float64) in Base at float.jl:399
[9] *(z::Complex{Bool}, x::Bool) in Base at complex.jl:282
[10] *(z::Complex{Bool}, x::Real) in Base at complex.jl:300
[11] *(::Missing, ::Missing) in Base at missing.jl:92
[12] *(::Missing, ::Number) in Base at missing.jl:93
[13] *(d::Missing, x::AbstractString) in Base at missing.jl:139
[14] *(this::Base.Grisu.Float, other::Base.Grisu.Float) in Base.Grisu at grisu/float.jl:140
[15] *(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:353
[16] *(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:443
[17] *(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:442
[18] *(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:441
[19] *(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:412
[20] *(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:460
[21] *(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:462
[22] *(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:503
[23] *(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:496
[24] *(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:490
[25] *(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:349
[26] *(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:318
[27] *(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:325
[28] *(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:333
[29] *(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:341
[30] *(B::BitArray{2}, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.0/LinearAlgebra/src/uniformscaling.jl:145
[31] *(y::AbstractFloat, x::Bool) in Base at bool.jl:120
[32] *(x::T, y::T) where T<:Union{Int128, UInt128} in Base at int.jl:788
[33] *(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:54
[34] *(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:461
[35] *(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:463
[36] *(a::Integer, b::Integer) in Base at int.jl:800
[37] *(x::Real, r::StepRangeLen{#s57,#s56,S} where S where #s56<:Base.TwicePrecision where #s57<:Real) in Base at twiceprecision.jl:479
[38] *(z::Complex, w::Complex) in Base at complex.jl:268
[39] *(z::Complex, x::Bool) in Base at complex.jl:289
[40] *(x::Real, z::Complex{Bool}) in Base at complex.jl:299
[41] *(x::Real, z::Complex) in Base at complex.jl:311
[42] *(z::Complex, x::Real) in Base at complex.jl:312
[43] *(x::Rational, y::Rational) in Base at rational.jl:259
[44] *(a::Integer, index::CartesianIndex{N}) where N in Base.IteratorsMD at multidimensional.jl:115
[45] *(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:329
[46] *(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:337
[47] *(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:345
[48] *(x::AbstractIrrational, y::AbstractIrrational) in Base at irrationals.jl:133
[49] *(y::Real, x::Dates.Period) in Dates at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:93
[50] *(x::Number) in Base at operators.jl:478
[51] *(x::T, y::T) where T<:Number in Base at promotion.jl:412
[52] *(x::Number, y::Number) in Base at promotion.jl:314
[53] *(x::Base.TwicePrecision{#s57} where #s57<:Union{Float16, Flo

The arguments a function can take can be restricted using the `::`-operator. Here, if we limit a function as `f(x::T)`, this means that `x` can be any type `<: T`. Can you explain the reasoning behind the following code? How does it process numbers? What does it do with strings?

In [28]:
twice(x::Number) = 2x;
twice(x::String) = x * x;

So why use dispatch?

1. Controls the **scope** of functions. For some types, there is no method, and we want an error if we use the wrong type.
2. Because we wish functions to **behave differently** depending on the types you feed into them.

Note that, generally seen, typing the functions is **not** needed to improve the performance of the functions.

> **Exercise** Consider the following methods. Can you predict the outcome of the lines of code below it?

In [29]:
f(x, y) = println("No life forms present");
f(x::T, y::T) where {T} = x * y;  # short for {T <: Any}
f(x::Integer, y::Real) = 2x + y;
f(x::Int, y::Int) = 2x + 2y;
f(x::Integer, y::Float64) = x + 2y;
f(x::Float64, y::Real) = x - y;
f(x::Float64, y::Float64) = 2x - y;

Gives

In [30]:
f(1, 2.0)

5.0

In [31]:
f(1.0, 2)

-1.0

In [32]:
f(Int8(1), Int8(2))

4

In [33]:
f(1.0, 2.0)

0.0

In [34]:
f("one", 2)

No life forms present


In [35]:
f("one", "two")

"onetwo"

In [36]:
f(1, Float32(2.0))

4.0f0

In [37]:
f(1, 2)

6

In [38]:
f([1 1; 1 1], [2.0 2.0; 2.0 2.0])

No life forms present


In [39]:
f([1 1; 1 1], [2 2; 2 2])

2×2 Array{Int64,2}:
 4  4
 4  4

# Defining types

## Abstract types

Abstract types are defined using the following simple syntax:

```
abstract type «name» end
abstract type «name» <: «supertype» end
```

## Primitive types

*Primitive types* exist of simple bits. Examples are `Float64` and `Int16`. You can declare your own types, though this is likely not something many often do in practice.

## Composite types

*Composite types* (records, structs, or objects) are more exciting. They are often containers for several objects set to behave in a certain way. Take the following small example of defining an abstract type `Pet`, which two concrete structs for cats and dogs, each has `name` as a sole attribute.

In [40]:
abstract type Pet end

struct Cat <: Pet
  name
end

struct Dog <: Pet
  name
end

# overload the method for showing the pets
Base.show(io::IO, pet::Pet) = print("pet $(pet.name)")

# what kind of sound does the pet make?
calls(cat::Cat) = println("miaaw")
calls(dog::Dog) = println("woof")

calls (generic function with 2 methods)

In [41]:
mycat = Cat("Appa");
hisdog = Dog("Storm");

In [42]:
calls(mycat)

miaaw


In [43]:
calls(hisdog)

woof


Using dispatch, we can have custom behavior for the pets.

In [44]:
meets(pet1::Cat, pet2::Dog) = println("$(pet1.name) hisses at $(pet2.name)");
meets(pet1::Dog, pet2::Cat) = println("$(pet1.name) barks at $(pet2.name)");
meets(pet1::Cat, pet2::Cat) = println("$(pet1.name) ignores $(pet2.name)");
meets(pet1::Dog, pet2::Dog) = println("$(pet1.name) sniffs the but of $(pet2.name)");

In [45]:
meets(mycat, hisdog)

Appa hisses at Storm


In [46]:
meets(hisdog, mycat)

Storm barks at Appa


## Union types

Two or more types can be combined into an `Union` type. For example, `Union{Cat, Dog}` would relate to either a cat or a dog.

## Parametric types

Julia allows for more control of the types. For example, consider a 2-dimensional coordinate:

In [47]:
struct Point{T}
    x::T
    y::T
end

So here, each coordinate of the type `Point` has two attributes, `x` and `y`, of the same type. These so-called *parametric types* allow for processing the structures the right way.

In [48]:
p = Point(1.0, 2.0)

Point{Float64}(1.0, 2.0)

In [49]:
Point(1, 2)

Point{Int64}(1, 2)

But what will happen if you evaluate `Point(1, 2.0)`?

In [50]:
Point(1, 2.0)

MethodError: MethodError: no method matching Point(::Int64, ::Float64)
Closest candidates are:
  Point(::T, !Matched::T) where T at In[47]:2

Parametric types can be used in dispatch. For example, if we want to compute the norm of a Point, this would only make sense if Point is of type real.

In [51]:
norm(p::Point{T} where {T<:Real}) = sqrt(p.x^2 + p.y^2);

In [52]:
norm(p)

2.23606797749979

# Constructors

## Outer constructors

Constructors are functions that create new objects. We have already seen that when creating a new `struc`, this immediately initiates the constructor (e.g., `Point(1.0, 2.0)`). These can also be made explicitly:

In [53]:
Point(x::T, y::T) where {T<:Real} = Point{T}(x,y);

Constructors, however, allow us to have custom behavior when initializing types. For example, we have seen that `Point(1, 2.0)` won't work, because the two inputs are of the same type. In this case, we can make the rule that one of the inputs has to be promoted to a more general type.

In [54]:
Point(x::Real, y::Real) = Point(promote(x, y)...);

In [55]:
Point(1, 2.0)

Point{Float64}(1.0, 2.0)

Or, suppose that we want that if only one input is provided, it just uses this twice, e.g. `Point(1)` behaves as `Point(1, 1)`:

In [56]:
Point(x) = Point(x, x);

Point(1)

Point{Int64}(1, 1)

## Inner constructors

The above examples show *outer constructors*. These are defined outside the structure. We can also use *inner constructors*, which are declared within the definition of the type. These make use of the keyword `new`. For example, let us define an ordered pair.

In [57]:
struct OrderedPair
  x
  y
  function OrderedPair(x, y)
    if x < y
      new(x, y)
    else
      new(y, x)
    end
  end
end

In [58]:
OrderedPair(18, 23)

OrderedPair(18, 23)

In [59]:
OrderedPair(8, 2)

OrderedPair(2, 8)

# Exercises

## Case study: the Strang matrix

The Strang matrix is a tridiagonal matrix with -2 at the diagonal and above and below the diagonal.

$$
\begin{pmatrix}
  -2 & 1& \cdots & 0 & 0 \\
  1 & -2& \cdots & 0 & 0\\
  \vdots  & \vdots  & \ddots & \vdots  \\
  0 &0 & \cdots & -2 & 1\\
0 &0 & \cdots & 1 & -2
 \end{pmatrix}
$$

The specific structure makes computing with this matrix often more straightforward than for general matrices. Let us implement a `Strang` matrix type to work with this!

In [60]:
struct Strang{T<:Number} <: AbstractMatrix{T}
  n::Int
  function Strang(T::Type,n::Integer)
    @assert n > 0 throw(AssertionError("n should be positive, got $n"))
    new{T}(n)
  end
end

Strang(n::Integer) = Strang(Float64, n)

Strang

In [61]:
Base.eltype(S::Strang{T})  where {T<:Real} = T
Base.size(S::Strang) = (S.n, S.n)
Base.getindex(S::Strang{T}, i, j)  where {T} = i==j ? -2one(T) : (abs(i-j)==1 ? one(T) : zero(T))

> **Optional assignment** Before running the code, modify the `struct` such that `Strang` is paramterically typed, with a constructor, for example, `Strang(Int, n)` generates an $n \times n$ Strang matrix with `Int` elements. The default version of `Strang(n)` generates a Strang matrix of type `Strang{Float64}`.

That's it! Now, we have our own implementation of the Strang matrix. We have also given Julia just enough information such that it can also represent this nicely!

In [62]:
S = Strang(5)

5×5 Strang{Float64}:
 -2.0   1.0   0.0   0.0   0.0
  1.0  -2.0   1.0   0.0   0.0
  0.0   1.0  -2.0   1.0   0.0
  0.0   0.0   1.0  -2.0   1.0
  0.0   0.0   0.0   1.0  -2.0

Note that much other functionality just magically appears.

In [63]:
sum(S)

-2.0

In [64]:
v = [2.3, 1.4, 6.0, 9.0 , 1.7]
S * v

5-element Array{Float64,1}:
  -3.1999999999999997
   5.5               
  -1.5999999999999996
 -10.3               
   5.6               

> **Assignment** Though multiplication works, it is quite slow, because Julia has no sense of the particular structure of Strang matrices. Overload the `*` function in Julia to provide efficient multiplication.

In [65]:
function Base.:*(S::Strang, v::Vector)
  x = -2v
    x[1:end-1] .+= v[2:end]
    x[2:end] .+= v[1:end-1]
  return x
end

In [66]:
S * v  # the same

5-element Array{Float64,1}:
  -3.1999999999999997
   5.5               
  -1.6               
 -10.3               
   5.6               

## Wizarding currency

The British wizarding world uses Galleons, Sickles, and Knuts as a currency. There are 17 Sickles in a Galleon, and 29 Knuts in a Sickle, meaning there are 493 Knuts to a Galleon. We will make a structure `WizCur` to represent wizarding currency. This structure has three integer-valued fields: `galleons`, `sickles`, and `knuts`. The constructor should always create tidy representations, meaning that, for example, if the number of knuts is 29 or more, it just adds an appropriate number of sickles such that the number knuts is less than 29 (it's magical money). The same applies to the sickles, which can also never exceed 17.

Overload `Base.show` such that Julia prints your currency as, for example, `7G, 2S, 9K`.

Also, overload the function `+` to add two instances of `WizCur` and the `>` and `<` operators to compare two instances of wizarding currency.

The piggy bank with Ron's life savings contains 19 Sickles and 732 Knuts. Harry has 3 Galleons, 1 Sickle, and 7 Knuts pocket change. Who has the most money? How many do they have together?

In [67]:
struct WizCur
  knuts::Int
  sickles::Int
  galleons::Int
  function WizCur(knuts::Int, sickles::Int, galleons::Int)
        knuts, sickles = knuts % 29, sickles + div(knuts, 29)
        sickles, galleons = sickles % 17, galleons + div(sickles, 17)
        new(knuts, sickles, galleons)
    end
end

Base.show(io::IO, wizcur::WizCur) = print("$(wizcur.knuts)K, $(wizcur.sickles)S, $(wizcur.galleons)G")


In [68]:
WizCur(0, 0, 30)

0K, 0S, 30G

In [69]:
Base.:+(c1::WizCur, c2::WizCur) = WizCur(c1.knuts+c2.knuts, c1.sickles+c2.sickles, c1.galleons+c2.galleons)
knuts(c::WizCur) = c.knuts + 17c.sickles + 493c.galleons
Base.:>(c1::WizCur, c2::WizCur) = knuts(c1) > knuts(c2)
Base.:<(c1::WizCur, c2::WizCur) = knuts(c1) < knuts(c2)

In [70]:
money_ron = WizCur(732, 19, 0)

7K, 10S, 2G

In [71]:
money_harry = WizCur(7, 1, 3)

7K, 1S, 3G

In [72]:
money_harry > money_ron

true